- 환경 변수 가져오기

In [1]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [2]:
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')
PINECONE_API_KEY

'pcsk_3RAn2d_6bkVakuPSMkobvFRxMZLjKBcmNWLGBdiLCAH9wvaDR6xFNKjvbCh8MgG7RQSU7H'

- Pinecone 클라이언트 초기화

In [3]:
from pinecone import Pinecone, ServerlessSpec
pine = Pinecone(api_key=PINECONE_API_KEY)
# 동일 : pine = Pinecone(api_key=getenv('PINECONE_API_KEY'))

- 인덱스 생성(서버리스)

In [4]:
index_name='wiki'

In [5]:
pine.create_index(
    name=index_name,
    dimension=1536,
    metric="cosine",
    spec=ServerlessSpec(cloud='aws',region='us-east-1')
)

PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2025-04', 'x-cloud-trace-context': 'e86bd285b9a6321985779fd41bee36ff', 'date': 'Wed, 12 Nov 2025 07:39:58 GMT', 'server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


- 임베딩 객체 생성

In [ ]:
# 사용할 인덱스 가져오기
index = pine.Index(index_name)
index.describe_index_stats()

c:\Users\C118\miniconda3\envs\llmEnv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


{'dimension': 1536,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {},
 'total_vector_count': 0,
 'vector_type': 'dense'}

In [ ]:
from langchain_openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings(model='text-embedding-3-small')
# 임베딩 모델의 차원(dimension) 과 생성한 인덱스의 dimension 이 같아야 합니다.

- 벡터DB 에 저장할 데이터셋 가져오기
  - 영어로 20231101 버전의 위키백과 600만개의 row 중 100개 
  - 데이터의 크기가 크면 임베딩 시간이 오래 걸림.

In [ ]:
from datasets import load_dataset
# pip install datasets
# dataset = load_dataset("wikimedia/wikipedia","20231101.en",split="train[:100]")  # 7GB 전체를 다운로드
dataset = load_dataset("parquet",data_files=["train-03.parquet"])

In [ ]:
# iterable 한 Dataset 타입 리턴
print(dataset) # num_rows : 156289개

DatasetDict({
    train: Dataset({
        features: ['column0'],
        num_rows: 471
    })
})


In [ ]:
data = dataset['train'][0:100]    # 여전히 dict
print(len(data))   # dict : 4개의 키값.4개의 키 안에 리스트 크기 100
print(data['text'][0])
# select() 는 dataset 함수
data = dataset['train'].select(range(100))    # dict 의 리스트
print(len(data))   # list 의 크기 : 100

['column0']
{'column0': '<s>[INST] What class is Aatrox? [/INST] Fighter </s>'}


In [ ]:

# print(data['text'][0])
print(data[85])

100
<s>[INST] What class is Aatrox? [/INST] Fighter </s>


- 청크
  - splitter 객체를 생성해서 문자열을 나누기

In [ ]:
from langchain_text_splitters import  RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(
  chunk_size = 400,  # 텍스트 분할 크기
  chunk_overlap=20, # 분할할 텍스트의 중첩 크기
  length_function=len,
  separators = ['\n', ' '] # 분할할 때 사용할 텍스트(단어) 구분자
)

- 청킹 후 임베딩 -> 업서트(batch_size 크기만큼 수행)

In [ ]:
from uuid import uuid4
import time

batch_size = 30
texts = []
metas = []
count = 0

# 데이터셋의 각 샘플에 대해 반복합니다.  features: ['id', 'url', 'title', 'text'],
for i,sample in enumerate(data):
    full_text = sample["text"] # Wikipedia 문서 텍스트 -> 청킹 후 임베딩
    metadata = {  # key 구성은 임의로 합니다. 청킹된 데이터의 소속을 구별
        'wiki_id': str(sample["id"]),  # Wikipedia 문서 ID
        'url': sample['url'],
        'title': sample["title"],  # Wikipedia 문서 제목
    }


    chunks = splitter.split_text(full_text)  # 텍스트를 청크로 분할합니다.
    print(len(chunks))  # full_text 를 몇개로 분할 했는지 확인

    # 각 청크에 대해 반복합니다.
    for i, chunk in enumerate(chunks):
        # 실제로 벡터db에 업서트할 record
        record = {
            'chunk_id': i,  # 청크 ID
            'head_chunk': chunk[:200],  # 전체 텍스트
            **metadata,  # 메타데이터 언패킹
        }

        texts.append(chunk)  # 청크(분할된 텍스트)를 텍스트 목록에 추가합니다.
        metas.append(record)  # 메타데이터를 메타데이터 목록에 추가합니다.

        count += 1  # 처리된 청크 수를 증가시킵니다.

        # batch_size만큼의 청크를 처리 : 임베딩 -> 업서트
        if count % batch_size == 0:
            # Pinecone 인덱스에 청크를 추가합니다.청크 text 갯수(batch_size) 만큼 레코드 uuid 만들기
            ids = [str(uuid4()) for _ in range(len(texts))]
            # 임베딩. 30개씩(batch_size).
            embeddings = embedding.embed_documents(texts)
            index.upsert(
                vectors=zip(ids, embeddings, metas),
                namespace="wiki-ns1")
            # 청크 목록과 메타데이터 목록을 비웁니다.
            texts = []
            metas = []
            # 1초 대기합니다.
            time.sleep(1)

data type: <class 'datasets.arrow_dataset.Dataset'>
Columns: ['text']
첫 샘플: {'text': '<s>[INST] What class is Aatrox? [/INST] Fighter </s>'}
자동으로 컬럼 선택: text


Processing texts: 100%|██████████| 100/100 [00:06<00:00, 14.46it/s]
